## One map (use techniques from Week 3 and 4)

In this part we will analyz crime data from May 2017 to May 2018, specifically focusing on larceny and theft incidents.

First, we imported the necessary libraries, including Folium and Pandas. Folium is a Python library that allows us to create interactive maps, and Pandas is a data manipulation library.

In [27]:
# Import the necessary libraries
import folium
import pandas as pd
from folium.plugins import HeatMapWithTime

Next, we importe the crime dataset into the Jupyter Notebook using Pandas. We also parse the date and time columns to create a single datetime column for easier data analysis.

In [28]:
# Import the dataset
df = pd.read_csv('../weeksex/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', parse_dates=[['Date', 'Time']])

C:\Users\Omare\AppData\Local\Temp\ipykernel_130684\2820924313.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../weeksex/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', parse_dates=[['Date', 'Time']])


In [37]:
# Filter the dataset to only include larceny and theft incidents between May 2017 and May 2018.
df_larceny_theft = df.loc[(df['Date_Time'] >= "2017-05-31") &
(df['Date_Time'] < "2018-05-31") &
(df['Category'] == "LARCENY/THEFT")].reset_index(drop=True)

In [38]:
# Create a list of latitude and longitude pairs for each incident
x_y = [(lat, lon) for lat, lon in zip(df_larceny_theft["Y"], df_larceny_theft["X"])]

We create a map of San Francisco using Folium. We set the map's initial coordinates and zoom level and chose the "Stamen Toner" tileset for the map's visual style.
To plot the crime incidents on the map, we loop through the dataset and create a circle marker for each crime incident. We set the marker's color to red and radius to 1 to distinguish them from each other. We add the markers to the map, creating an interactive visualization of the crime incidents in San Francisco.

In [39]:
# Plot the locations on a map
SF_map = folium.Map(location=[37.77919, -122.41914], zoom_start=12, tiles="Stamen Toner")
folium.Marker(location=[37.77919, -122.41914], popup='SF City Hall').add_to(SF_map)

for location in x_y:
    folium.CircleMarker(location=location, radius=1, color='red').add_to(SF_map)

#SF_map

In [41]:
# Create a column in the dataset that indicated the week number for each incident.
df_larceny_theft["week number"] = df_larceny_theft["Date_Time"].dt.week

AttributeError: Can only use .dt accessor with datetimelike values

In [42]:
# Group the incidents by week number and count the number of larceny and theft incidents per week.
larceny_theft_pr_week = df_larceny_theft.groupby(by=["week number"]).size().reset_index(name="larceny/theft pr. week")

KeyError: 'week number'

Using the latitude and longitude data from the filtered dataset, we create a heatmap for each week showing the density of larceny and theft incidents in each location.

In [ ]:
# Create a list of (latitude, longitude) pairs for each week
heatmap = []
for i in range(1, 19):
    heatmap_1 = []
    
    for _, row in df_larceny_theft[df_larceny_theft["week number"] == i].iterrows():
        lat_lon = [row["Y"], row["X"]]
        heatmap_1.append(lat_lon)
    
    heatmap.append(heatmap_1)

We add the heatmap to the Folium map we had created earlier.

In [ ]:
# Create a map and add the heat map
SF_map = folium.Map(location=[37.77919, -122.41914], zoom_start=12, tiles="Stamen Toner")
HeatMapWithTime(heatmap, auto_play=True, max_opacity=0.7).add_to(SF_map)

#SF_map

Lastly, we add markers to the map for each police district in San Francisco, using the mean latitude and longitude of the incidents in each district. We includ a popup for each marker displaying the district name and the number of crimes committed in that district during the given time frame.

In [ ]:
district_counts = df["PdDistrict"].value_counts()

# Add a marker for each police district
for district in district_counts.index:
    # Get the latitude and longitude of the district
    district_data = df[df["PdDistrict"] == district]
    latitude = district_data["Y"].mean()
    longitude = district_data["X"].mean()
    
    # Add a marker for the district
    folium.Marker(
        location=[latitude, longitude],
        popup=f"{district}: {district_counts[district]} crimes",
        icon=folium.Icon(color="purple", icon="info-sign")
    ).add_to(SF_map)

In [ ]:
# Display the map
SF_map

In conclusion, by utilizing the map, and analyzing the concentration of crime in different police districts, we have obtained valuable insights that can help identify patterns and trends in the data. This can aid in making recommendations to improve public safety in San Francisco and enable the Police to gain a better understanding of crime trends in the city.